In [5]:
import boto3
#import sagemaker

import numpy as np
from PIL import Image
import io
import numpy as np
import json
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [6]:
print(boto3.__version__)

1.34.153


In [7]:
#client = boto3.client('sagemaker-runtime')
AWS_ACCESS_KEY = ''
AWS_SECRET_KEY = ''
AWS_SESSION_TOKEN = ""
AWS_REGION_NAME = ''
# AWS_S3_BUCKET_NAME = "s3-avalanche-guard"
AWS_S3_BUCKET_NAME = "avlancheguard-uploaded-images"

client = boto3.client(
    'sagemaker-runtime',
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION_NAME

    # aws_session_token=AWS_SESSION_TOKEN
)


In [ ]:
modelnumber = 1


if modelnumber == 1:
    endpoint_name = 'tensorflow-inference-2024-08-05-06-14-21-104'

elif modelnumber == 2:

    endpoint_name = 'tensorflow-inference-2024-08-05-06-24-06-679'

elif modelnumber == 4:

    endpoint_name = 'tensorflow-inference-2024-08-05-06-30-40-683'



count=0
LIMIT = 5
directory = '../temp/'
image_size = (224,224)


for filename in os.listdir(directory):
    count=count+1
    if count > LIMIT:
        break
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):

        # Load the image
        img = Image.open(f)
        # Resize the image while maintaining the aspect ratio
        img.thumbnail(image_size)
        # Convert the image to a byte array
        img_byte_array = np.expand_dims(img, 0)  # Create batch axis
        body = json.dumps(img_byte_array.tolist())
        content_type = 'application/json'
        try:
            ioc_response = client.invoke_endpoint(
                EndpointName=endpoint_name,
                Body=body,
                ContentType=content_type,

             )
            response = json.loads(ioc_response['Body'].read())

            if modelnumber == 1:
                prediction_score_raw = list(response["predictions"])[0][0]
                prediction_score = prediction_score_raw

                print(f"{100 * prediction_score:.2f}% Terrain. Raw prediction score={prediction_score_raw}")
            elif modelnumber == 2:
                prediction_score_raw = list(response["predictions"])[0][0]
                prediction_score =  prediction_score_raw

                print(f"{100 * prediction_score:.2f}% Avalanche.")
            elif modelnumber == 4:
                print(response)
                prediction_score_raw = list(response["predictions"])[0]
                glide = prediction_score_raw[0] * 100
                loose = prediction_score_raw[1] * 100
                slab = prediction_score_raw[2] * 100

                ind = 0
                max_element = prediction_score_raw[0]
                for i in range (1,len(prediction_score_raw)): #iterate over array
                    if prediction_score_raw[i] > max_element: #to check max value
                        max_element = prediction_score_raw[i]
                        ind = i

                if ind == 0:
                    avlType = 'Glide'
                elif ind == 1:
                    avlType = 'Loose'
                elif ind == 2:
                    avlType = 'Slab'
                #prediction_score = 1/(1+np.exp(-prediction_score_raw) )

                print(f"Avalanche Type={avlType}, {100 * prediction_score_raw[ind]:.2f}%, All Values [Glide Loose Slab]={prediction_score_raw}%")


            # Plot the image
            plt.figure(figsize=(6, 3))
            plt.imshow(img)
            plt.axis('off')  # Hide the axis
            plt.show()
        except Exception as error:
            print(f"error infering {error}")
        finally:
            pass



In [ ]:
import boto3
import botocore
import urllib3
print("boto3 version:", boto3.__version__)
print("botocore version:", botocore.__version__)
print("urllib3 version:", urllib3.__version__)
print("Pillow Image version: ",Image.__version__)